## Universidade Federal do Rio Grande do Sul (UFRGS)   
Programa de Pós-Graduação em Engenharia Civil (PPGEC)   

# Dinâmica de Sistemas Estruturais


### Análise de Treliça Plana (Teoria e codificação em Python3)



[1.   Apresentando o Problema](#section_1)  
[2.   Apresentando o Código](#section_2)  
[3.   Resultados](#section_3)  
[4.   Validação](#section_4)  
[5.   Referências](#section_5)  

---
_Mestrando. Alex Koch de Almeida._ [(Lattes)](http://lattes.cnpq.br/7734327250758963)  
_Porto Alegre, RS, Brazil_ 


In [1]:
#Importando os módulos do Python a serem utilizados pelo código
#Esta célula deve ser executada antes de qualquer outra
import numpy as np 
from numpy.linalg import norm
from scipy.linalg import eigh
import matplotlib.pyplot as plt 

## 1. Apresentando o Problema <a name="section_1"></a> 

Na figura abaixo está representada a treliça plana a ser utilizada neste artigo:

<img src="img/img1.png" width="460px"/>

Primeiramente vamos definir a notação a ser utilizada para a descrição geométrica do problema: 

<img src="img/img2.png" width="460px"/>

Onde:

$N$: nó próximo, ou seja, nó inicial da descrição geométrica onde posicionam-se os eixos X e Y Globais &nbsp;

$F$: nó afastado, ou seja, nó final da descrição geométrica &nbsp;

$\theta_x$: ângulo iniciado no eixo $X$ (global) e finalizado no eixo $X_L$ (local), em sentido anti-horário&nbsp;


$\theta_y$: ângulo iniciado no eixo $X_L$ (local) e finalizado no eixo $Y$ (global), em sentido anti-horário&nbsp;

**Adotando portanto, para o problema proposto a seguinte configuração:**&nbsp;

Barra 1: N = 1; F = 3&nbsp;

Barra 2: N = 2; F = 3&nbsp;

Barra 3: N = 1; F = 2&nbsp;

**Unidades a serem utilizadas:**&nbsp;

Comprimento = m&nbsp;

Força = kN&nbsp;

**Método a ser utilizado:**&nbsp;

Método dos deslocamentos

## 2. Apresentando o Código <a name="section_2"></a> 

O presente código está organizado em uma rotina **principal** do tipo `def` que divide-se em uma ordem lógica de processos e, dentro de cada processo, existem sub-rotinas do tipo `def` que de fato manipulam os dados e resolvem o problema:

O primeiro conjunto de rotinas objetivam descrever geometricamente o problema de forma legível e manipulável pelo Python:

O próximo conjunto de rotinas busca retornar os cossenos diretores necessários para rotacionar as matrizes locais, usando a função `np.dot()` que faz o produto escalar entre dois vetores e a função `norm` que retorna o comprimento de um vetor. Observe que as barras do problema, denominadas "elemento", são representadas por um vetor com quatro coordendas e a função `norm` retorna o comprimento da Barra descrita por esse vetor. As equações a serem utilizadas, da geometria analítica, são as seguintes:&nbsp;

$$ cos\theta = \frac{\vec{Elemento} \; \vec{e_x}}{\vec{|Elemento|} \; |\vec{e_x}|} $$

$$ cos\phi = sen\theta= \frac{\vec{Elemento} \; \vec{e_y}}{\vec{|Elemento|} \; |\vec{e_y}|} $$

onde, 

<img src="img/img3.png" width="320px"/>

Sendo,

$\vec{e_x}$: Vetor unitário definido anteriormente em `def config()`como `x_eixo` &nbsp;

$\vec{e_y}$: Vetor unitário definido anteriormente em `def config()`como `y_eixo` &nbsp;

A rotina da próxima célula, `matrizes()`, é de fato a mais importante desde código e, para entendê-la por completo, a teoria embasadora será apresentada a seguir: &nbsp;

Um elemento de treliça desloca-se apenas ao longo do seu eixo $X_L$(local) uma vez que as cargas são aplicadas ao longo desse eixo. Sendo assim, apenas dois deslocamentos são possíveis.&nbsp;

Quando um deslocamento positivo $u_N$ é imposto ao nó próximo $N$ do elemento enquanto o nó afastado $F$ é mantido fixo, as forças desenvolvidas nos nós $N$ e $F$ são:

$$p'_N = \frac{AE}{L} \; u_N \;\;\;\;\;\;\;p'_F = -\frac{AE}{L} \; u_N $$

<img src="img/img4.png" width="320px"/>

Onde $p'_F$ é negativo, uma vez que ele atua no sentido $X_L$ negativo, mantendo o equilibrio.&nbsp;

Da mesma forma, quando um deslocamento positivo $u_F$ é imposto ao nó afastado $F$ do elemento enquanto o nó próximo $N$ é mantido fixo, as forças desenvolvidas nos nós $N$ e $F$ são:

$$p''_N = -\frac{AE}{L} \; u_F \;\;\;\;\;\;\;p''_F = \frac{AE}{L} \; u_F $$

<img src="img/img5.png" width="320px"/>

Onde $p''_N$ é negativo, uma vez que ele atua no sentido $X_L$ negativo, mantendo o equilibrio.&nbsp;

Sendo assim, por superposição, as forças resultantes causadas por ambos os deslocamentos são:

$$p_N = \frac{AE}{L} \; u_N - \frac{AE}{L} \; u_F$$ &nbsp;

$$p_N = -\frac{AE}{L} \; u_N + \frac{AE}{L} \; u_F$$

<img src="img/img6.png" width="320px"/>

Que pode ser reescrito matricialmente da seguinte forma:

$$\left [ \begin{array}{cc}
           p_N \\
           p_F
          \end{array} \right ] =            
          \frac{AE}{L}            
  \left [ \begin{array}{cc}
           1 & -1 \\
           -1 & 1
          \end{array} \right ]          
  \left [ \begin{array}{cc}
           u_N \\
           u_F
          \end{array} \right ]$$

Ou ainda:

$$p=ku$$

onde,

$$k = \frac{AE}{L} 
      \left [ \begin{array}{cc}
               1 & -1 \\
               -1 & 1
              \end{array} \right ]$$

$k$ é denominada **matriz de rigidez local**, ou ainda **matriz de rigidez do elemento**&nbsp;

Para que $k$ de cada elemento possa ser utilizada na resolução do problema, ou seja, compor a matriz de rigidez global $K$, a mesma deve ser rotacionada dos seus eixos locais para os eixos globais. Para tanto, utiliza-se a **matriz de rotação** $T$:

$$T = \left [ \begin{array}{cc}
               \lambda_x & \lambda_y & 0 &0 \\
               0 & 0 & \lambda_x & \lambda_y
              \end{array} \right ]$$

onde,
$$\lambda_x=cos\theta_x\;\;\;\;\;\;\;\lambda_y=sen\theta_x$$

Em coordendas globais cada Nó do elemento pode ter dois graus de liberdade (ou deslocamentos independentes), o nó $N$ terá $U_{Nx}$ e $U_{Ny}$, o nó $F$ terá $U_{Fx}$ e $U_{Fy}$.&nbsp;

Quando um deslocamento global $U_{Nx}$ é imposto ao nó próximo $N$ do elemento, o deslocamento correspondente ao longo do elemento é dado por $U_{Nx}\lambda_x$.

<img src="img/img7.png" width="320px"/>

Da mesma forma, quando um deslocamento global $U_{Ny}$ é imposto ao nó próximo $N$ do elemento, o deslocamento correspondente ao longo do elemento é dado por $U_{Ny}\lambda_y$.

<img src="img/img8.png" width="320px"/>

O efeito de ambos os deslocamentos globais faz que o elemento seja deslocado: 
$$u_N=U_{Nx}\lambda_x+U_{Ny}\lambda_y$$

De maneira análoga, o mesmo ocorre para deslocamentos impostos em $F$:

<img src="img/img9.png" width="320px"/> 
<img src="img/img10.png" width="320px"/>

E portanto:

$$u_F=U_{Fx}\lambda_x+U_{Fy}\lambda_y$$

Que pode ser reescrito matricialmente da seguinte forma:

$$\left [ \begin{array}{cc}
           u_N \\
           u_F
          \end{array} \right ] = 
  \left [ \begin{array}{cc}
           \lambda_x & \lambda_y & 0 & 0 \\
           0 & 0 & \lambda_x & \lambda_y
          \end{array} \right ]
  \left [ \begin{array}{cccc}
           U_{Nx} \\
           U_{Ny} \\
           U_{Fx} \\
           U_{Fy}
          \end{array} \right ]$$

Ou ainda:
$$u=TU$$

De maneira análoga aos deslocamentos, com a aplicação de cargas ocorre o mesmo, dessa forma:

$$P_{Nx}=p_N\lambda_x\;\;\;\;\;\;\;\;P_{Ny}=p_N\lambda_y$$
$$P_{Fx}=p_F\lambda_x\;\;\;\;\;\;\;\;P_{Fy}=p_F\lambda_y$$

Que pode ser reescrito matricialmente da seguinte forma:

$$\left [ \begin{array}{cccc}
           P_{Nx} \\
           P_{Ny} \\
           P_{Fx} \\
           P_{Fy}
          \end{array} \right ] = 
  \left [ \begin{array}{cccc}
           \lambda_x & 0 \\
           \lambda_y & 0 \\
           0 & \lambda_x \\
           0 & \lambda_y
          \end{array} \right ]
  \left [ \begin{array}{cc}
           p_{N}\\
           p_{F}
          \end{array} \right ]$$

Ou ainda:
$$P=T^tp$$

Considerando portanto as deduções anteriores teremos as seguintes equações finais:

$$p=kTU$$

$$k_{r}=T^tkT$$

$$P=k_{r}U$$

Onde $k_{r}$ é a **matriz de rigidez do elemento rotacionada para coordenadas globais**, e pode ser escrita portanto, matricialmente:

$$ k_{r} = \left [ \begin{array}{cccc}
                    \lambda_x & 0 \\
                    \lambda_y & 0 \\
                    0 & \lambda_x \\
                    0 & \lambda_y
                   \end{array} \right ] 
           \frac{AE}{L} 
           \left [ \begin{array}{cc}
                    1 & -1 \\
                    -1 & 1 
                   \end{array} \right ]
           \left[ \begin{array}{cc}
                   \lambda_x & \lambda_y & 0 & 0 \\
                   0 & 0 & \lambda_x & \lambda_y
                  \end{array} \right ]$$

onde,
$$Ck=\frac{AE}{L}$$

Definiremos agora uma matriz auxiliar $B$. Dado um vetor de deslocamentos Globais:

$$\vec{U}= \left[ \begin{array}{cccccc}
                   u_1 \\
                   v_1 \\
                   u_2 \\
                   v_2 \\
                   u_3 \\
                   v_3
                  \end{array} \right ]$$

Um elemento definido pelo nó $2$ e pelo nó $3$ terá seus deslocamentos dados por $u_2$,$v_2$,$u_3$,$v_3$. Sendo assim, o vetor de deslocamentos locais $\vec{u}$ deste elemento pode ser escrito matricialmente da seguinte forma:

$$\vec{u}= \left [ \begin{array}{cccc}
                    0 & 0 & 1 & 0 & 0 & 0 \\
                    0 & 0 & 0 & 1 & 0 & 0 \\
                    0 & 0 & 0 & 0 & 1 & 0 \\
                    0 & 0 & 0 & 0 & 0 & 1
                   \end{array} \right ]
           \left [ \begin{array}{cccccc}
                    u_1 \\
                    v_1 \\
                    u_2 \\
                    v_2 \\
                    u_3 \\
                    v_3
                   \end{array} \right ]=
           \left [ \begin{array}{cccc}
                    u_2 \\
                    v_2 \\
                    u_3 \\
                    v_3
                   \end{array} \right ]$$

Ou ainda:
$$\vec{u}=B\vec{U}$$

Defindo-se portanto, para este elemento em específico:

$$B = \left [ \begin{array}{cccc}
               0 & 0 & 1 & 0 & 0 & 0 \\
               0 & 0 & 0 & 1 & 0 & 0 \\
               0 & 0 & 0 & 0 & 1 & 0 \\
               0 & 0 & 0 & 0 & 0 & 1
              \end{array} \right ]$$

Dessa forma, para cada elemento do Problema, a matriz $B$ assumirá um formato diferente, alterando as posições dos valores não nulos em função da posição dos deslocamentos no vetor $\vec{U}$.

Isso implica que a posição Global das rigidezes de um elemento é dada por:

$$k_{rG}=B^tk_{r}B$$

É importante ainda destacar que a **matriz de rigidez global** $K$ a nivel de estrutura, é o somatório das diversas matrizes dos elementos nas posições definidas pelas conetividades das suas extremidades.&nbsp; 

$K$ é simétrica e sua ordem é dada pelo número de graus de liberdade do problema. No exemplo deste artigo teremos então uma matriz 6x6 (03 nós, com 02 graus de liberdade por nó). A teoria e a manipulação matricial envolvidas na montagem da matriz de rigidez global da estrutura $K$ podem ser consultadas na bibliografia mencionada em [Referências](#section_4).


Por fim, para análise dinâmica, define-se a **matriz de massa local** $m$:

$$m = \frac{\rho AL}{6} 
      \left [ \begin{array}{cc}
               2 & 1 \\
               1 & 2
              \end{array}\right]$$

onde,
$$Cm=\frac{\rho AL}{6}$$

Sobre $m$ incidirão as mesmas operações matriciais demonstradas anteriormente para $k$.

A próxima rotina é usada para definir as forças nos elementos, que neste caso, para treliças planas, são apenas axiais, de tração ou compressão.

## 3. Resultados <a name="section_3"></a> 
Uma função específica é criada onde iremos elencar os resultados a exibir:

In [25]:
def exibir_resultados(U , f_axial, freq):
    print ('Deslocamentos Nodais:', U)
    print ('Força axial', f_axial)
    print ('Frequências Naturais de Vibração:', freq)    

Finalmente, a Função principal, que representa a espinha dorsal do código, é definida. A Função principal será a primeira a ser executada e será responsável pela chamada de outras funções que por sua vez podem ou não chamar outras funções (e sucessivamente):

## 4. Validação <a name="section_4"></a> 

Para validar os resultados utilizaremos o `Ftool`, um software de análise estrutural 2D reconhecido no meio técnico cuja metodologia de cálculo é similar à utilizada neste artigo.&nbsp; 

Na figura abaixo deslocamentos no nó 2:

<img src="img/img11.png" width="720px"/>

Na figura abaixo deslocamentos no nó 3:

<img src="img/img12.png" width="720px"/>

Na figura abaixo forças axiais nos elementos:

<img src="img/img13.png" width="720px"/>

## 5. Referências<a name="section_5"></a> 

Hibbeler, R. C. Análise das estruturas. São Paulo: Pearson Education do Brasil, 2013.&nbsp;

https://github.com/apf99/Truss-Modeling/blob/master/truss.py &nbsp;

https://panda.ime.usp.br/panda/python/index